In [3]:
# installing the libraries for transformers
#!pip install -U -q sentence-transformers transformers bitsandbytes accelerate sentencepiece

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Deep Learning library
import torch

# to load transformer models
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline # T5 is Google

# to split the data
from sklearn.model_selection import train_test_split

# to compute performance metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


C:\Users\billl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))
misconceptions_filename = files[1]
train_filename = files[2]
test_filename = files[3]
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

IndexError: list index out of range

## File and Field Information
### [train/test].csv

* QuestionId - Unique question identifier (int).
* ConstructId - Unique construct identifier (int) .
* ConstructName - Most granular level of knowledge related to question (str).
* CorrectAnswer - A, B, C or D (char).
* SubjectId - Unique subject identifier (int).
* SubjectName - More general context than the construct (str).
* QuestionText - Question text extracted from the question image using human-in-the-loop OCR (str) .
* Answer[A/B/C/D]Text - Answer option A text extracted from the question image using human-in-the-loop OCR (str).
* Misconception[A/B/C/D]Id - Unique misconception identifier (int). Ground truth labels in train.csv; your task is to predict these labels for test.csv.

### misconception_mapping.csv
maps MisconceptionId to its MisconceptionName

### sample_submission.csv
A submission file in the correct format.
* QuestionId_Answer - Each question has three incorrect answers for which need you predict the MisconceptionId.
* MisconceptionId - You can predict up to 25 values, space delimited.

In [19]:
train_data = pd.read_csv(train_filename)
train_data_orig = train_data.copy()
train_data.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0


In [29]:
print ("train_data shape")
print("rows",train_data.shape[0])
print("columns",train_data.shape[1])

train_data shape
rows 1869
columns 15


In [32]:
subject_groups = train_data.groupby(["SubjectId","SubjectName"]).count().sort_values('QuestionId',ascending=False).reset_index()
print(subject_groups.shape[0], " different subject groups")
subject_groups.head()

163  different subject groups


,SubjectId,SubjectName,QuestionId,ConstructId,ConstructName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,64,Linear Equations,53,53,53,53,53,53,53,53,53,23,32,30,27
1,171,Linear Sequences (nth term),44,44,44,44,44,44,44,44,44,28,20,21,19
2,33,BIDMAS,37,37,37,37,37,37,37,37,37,20,21,23,24
3,65,Quadratic Equations,36,36,36,36,36,36,36,36,36,18,17,25,14
4,75,Area of Simple Shapes,36,36,36,36,36,36,36,36,36,22,26,23,24


In [5]:
print("percentages of null values")
pd.DataFrame({'Count':train_data.isnull().sum()[train_data.isnull().sum()>0],'Percentage':(train_data.isnull().sum()[train_data.isnull().sum()>0]/train_data.shape[0])*100})

percentages of null values


NameError: name 'pd' is not defined

In [ ]:
misc_con_data = pd.read_csv(misconceptions_filename)
misc_con_data.head()

In [ ]:
# defining a function to compute the cosine similarity between two embedding vectors
def cosine_score(text):
    # encoding the text
    embeddings = model.encode(text)

    # calculating the L2 norm of the embedding vector
    norm1 = np.linalg.norm(embeddings[0])
    norm2 = np.linalg.norm(embeddings[1])

    # computing the cosine similarity
    cosine_similarity_score = ((np.dot(embeddings[0],embeddings[1]))/(norm1*norm2))

    return cosine_similarity_score